**Import the main libraries**

In [58]:
import pandas as pd
import numpy as np

_import the local library_

**Import the Dataset**

In [59]:
## file path: windows style
data = pd.read_csv('productivity.csv')

# shape method gives the dimensions of the dataset
print('Dataset dimensions: {} rows, {} columns'.format(
    data.shape[0], data.shape[1]))

Dataset dimensions: 1197 rows, 15 columns


In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   date                   1197 non-null   object 
 1   quarter                1197 non-null   object 
 2   department             1197 non-null   object 
 3   day                    1197 non-null   object 
 4   team                   1197 non-null   int64  
 5   targeted_productivity  1197 non-null   float64
 6   smv                    1197 non-null   float64
 7   wip                    691 non-null    float64
 8   over_time              1197 non-null   int64  
 9   incentive              1197 non-null   int64  
 10  idle_time              1197 non-null   float64
 11  idle_men               1197 non-null   int64  
 12  no_of_style_change     1197 non-null   int64  
 13  no_of_workers          1197 non-null   float64
 14  actual_productivity    1197 non-null   float64
dtypes: f

In [35]:
data.head()

,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,1/1/2015,Quarter1,sewing,Thursday,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725
1,1/1/2015,Quarter1,finishing,Thursday,1,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.886500
2,1/1/2015,Quarter1,sewing,Thursday,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
3,1/1/2015,Quarter1,sewing,Thursday,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
4,1/1/2015,Quarter1,sewing,Thursday,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,0.800382


***
**Data Preparation and EDA** (unique to this dataset)
* _Check for missing values_
* _Quick visual check of unique values_
* _Split the classification feature out of the dataset_
* _Check column names of categorical attributes ( for get_dummies() )_
* _Check column names of numeric attributes ( for Scaling )_

**Check for missing values**

In [36]:
# Count the number of missing values for each column
data.isnull().sum()
total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, 
                         keys=['Total', 'Percent'])

,Total,Percent
wip,506,0.422723
date,0,0.000000
quarter,0,0.000000
department,0,0.000000
day,0,0.000000
team,0,0.000000
targeted_productivity,0,0.000000
smv,0,0.000000
over_time,0,0.000000
incentive,0,0.000000


In [37]:
# Remove one more column that will not help predict the outcome
data.drop(['wip'], axis=1, inplace=True)
data.head(2)

,date,quarter,department,day,team,targeted_productivity,smv,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,1/1/2015,Quarter1,sewing,Thursday,8,0.80,26.16,7080,98,0.0,0,0,59.0,0.940725
1,1/1/2015,Quarter1,finishing,Thursday,1,0.75,3.94,960,0,0.0,0,0,8.0,0.886500


**Quick visual check of unique values, deal with unique identifiers**

In [38]:
# Identify columns with only one value 
# or with number of unique values == number of rows
n_eq_one = []
n_eq_all = []
 
print('Unique value count (',data.shape[0],'Rows in the dataset )')
for col in data.columns:
    lc = len(data[col].unique())
    print(col, ' ::> ', lc)
    if lc == 1:
        n_eq_one.append(data[col].name)
    if lc == data.shape[0]:
        n_eq_all.append(data[col].name)

Unique value count ( 1197 Rows in the dataset )
date  ::>  59
quarter  ::>  5
department  ::>  3
day  ::>  6
team  ::>  12
targeted_productivity  ::>  9
smv  ::>  70
over_time  ::>  143
incentive  ::>  48
idle_time  ::>  12
idle_men  ::>  10
no_of_style_change  ::>  3
no_of_workers  ::>  61
actual_productivity  ::>  879


In [39]:
# Remove one more column that will not help predict the outcome
data.drop(['date'], axis=1, inplace=True)
data.head(2)

,quarter,department,day,team,targeted_productivity,smv,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,Quarter1,sewing,Thursday,8,0.80,26.16,7080,98,0.0,0,0,59.0,0.940725
1,Quarter1,finishing,Thursday,1,0.75,3.94,960,0,0.0,0,0,8.0,0.886500


* let's "bin" Age, Balance, and EstimatedSalary<br> 
_we could try to guess at the categories,_<br> 
_but let's use a function from our local library_

In [40]:
from mylib import autobin

In [41]:
for feat in ['over_time', 'incentive', 'no_of_workers']:
    print(feat, '- minValue: ', data[feat].min(), 
          '  maxValue: ', data[feat].max())

over_time - minValue:  0   maxValue:  25920
incentive - minValue:  0   maxValue:  3600
no_of_workers - minValue:  2.0   maxValue:  89.0


In [42]:
for feat in ['over_time', 'incentive', 'no_of_workers']:
    cuts, cats = autobin(data, feat)

    newcol = feat + 'Range'
    data[newcol] = pd.cut(data[feat], bins=cuts, labels=cats) 
    
    print('Unique value count:',feat,len(data[feat].unique()),
          '  ',newcol,len(data[newcol].unique())-1,'\n')

Found 9 Bins
Adding a Zero bin
Unique value count: over_time 143    over_timeRange 8 

Found 6 Bins
Adding a Zero bin
Unique value count: incentive 48    incentiveRange 6 

Found 7 Bins
Unique value count: no_of_workers 61    no_of_workersRange 7 



In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   quarter                1197 non-null   object  
 1   department             1197 non-null   object  
 2   day                    1197 non-null   object  
 3   team                   1197 non-null   int64   
 4   targeted_productivity  1197 non-null   float64 
 5   smv                    1197 non-null   float64 
 6   over_time              1197 non-null   int64   
 7   incentive              1197 non-null   int64   
 8   idle_time              1197 non-null   float64 
 9   idle_men               1197 non-null   int64   
 10  no_of_style_change     1197 non-null   int64   
 11  no_of_workers          1197 non-null   float64 
 12  actual_productivity    1197 non-null   float64 
 13  over_timeRange         1197 non-null   category
 14  incentiveRange         1197 non-null   c

In [44]:
data.head(5)

,quarter,department,day,team,targeted_productivity,smv,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity,over_timeRange,incentiveRange,no_of_workersRange
0,Quarter1,sewing,Thursday,8,0.80,26.16,7080,98,0.0,0,0,59.0,0.940725,6480-8639,Under 654,56-77
1,Quarter1,finishing,Thursday,1,0.75,3.94,960,0,0.0,0,0,8.0,0.886500,Under 2160,(Zero),2-11
2,Quarter1,sewing,Thursday,11,0.80,11.41,3660,50,0.0,0,0,30.5,0.800570,2160-4319,Under 654,23-33
3,Quarter1,sewing,Thursday,12,0.80,11.41,3660,50,0.0,0,0,30.5,0.800570,2160-4319,Under 654,23-33
4,Quarter1,sewing,Thursday,6,0.80,25.90,1920,50,0.0,0,0,56.0,0.800382,Under 2160,Under 654,45-55


In [45]:
# let's drop the detail and keep the categories
#   Using inPlace makes permanent changes to the dataframe in memory 
#   otherwise drop() will not affect the dataset we are working on
for feat in ['over_time', 'incentive', 'no_of_workers']:
    data.drop([feat], axis=1, inplace=True)

In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   quarter                1197 non-null   object  
 1   department             1197 non-null   object  
 2   day                    1197 non-null   object  
 3   team                   1197 non-null   int64   
 4   targeted_productivity  1197 non-null   float64 
 5   smv                    1197 non-null   float64 
 6   idle_time              1197 non-null   float64 
 7   idle_men               1197 non-null   int64   
 8   no_of_style_change     1197 non-null   int64   
 9   actual_productivity    1197 non-null   float64 
 10  over_timeRange         1197 non-null   category
 11  incentiveRange         1197 non-null   category
 12  no_of_workersRange     1191 non-null   category
dtypes: category(3), float64(4), int64(3), object(3)
memory usage: 98.2+ KB


**<br>Classification target feature**
<br>"the Right Answers", or more formally "the desired outcome"
<br>Must be in a separate dataset for classification ,,,

* Split the classification feature out of the dataset 

In [47]:
## Feature being predicted ("the Right Answer")
labels_col = 'actual_productivity'
y = data[labels_col]

## Features used for prediction 
# pandas has a lot of rules about returning a 'view' vs. a copy from slice
# so we force it to create a new dataframe 
X = data.copy()
X.drop(labels_col, axis=1, inplace=True)

**<br>Check column names of categorical attributes**
<br>Features with text values (categorical attributes) need to be normalised
<br>by changing them to numeric types that the algorithms find easier to work with

In [48]:
categori = X.select_dtypes(include=['object','category']).columns
print(categori.to_list())

['quarter', 'department', 'day', 'over_timeRange', 'incentiveRange', 'no_of_workersRange']


In [49]:
# check the distribution of the feature values 
for col in categori:
    print('Distribution of categories in', col)
    print(X[col].value_counts())
    print()

Distribution of categories in quarter
quarter
Quarter1    360
Quarter2    335
Quarter4    248
Quarter3    210
Quarter5     44
Name: count, dtype: int64

Distribution of categories in department
department
sewing       691
packing      330
finishing    176
Name: count, dtype: int64

Distribution of categories in day
day
Wednesday    208
Sunday       203
Tuesday      201
Thursday     199
Monday       199
Saturday     187
Name: count, dtype: int64

Distribution of categories in over_timeRange
over_timeRange
Under 2160     419
6480-8639      252
2160-4319      191
8640-10799     156
4320-6479      139
(Zero)          31
12960-15119      5
10800-12959      3
Above 23759      1
15120-23759      0
Name: count, dtype: int64

Distribution of categories in incentiveRange
incentiveRange
(Zero)        604
Under 654     583
654-980         5
981-1308        2
1309-2617       1
2618-3271       1
Above 3271      1
Name: count, dtype: int64

Distribution of categories in no_of_workersRange
no_of_worke

* 'one hot' encoding transforms a single column of text values into 
multiple columns of discrete values: 
it creates a new column for each unique value and puts
(one) in the column for which it is true and (zero) in the others

In [50]:
# the automatic way adds the original feature name
X = pd.get_dummies(X)

In [51]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 44 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   team                         1197 non-null   int64  
 1   targeted_productivity        1197 non-null   float64
 2   smv                          1197 non-null   float64
 3   idle_time                    1197 non-null   float64
 4   idle_men                     1197 non-null   int64  
 5   no_of_style_change           1197 non-null   int64  
 6   quarter_Quarter1             1197 non-null   bool   
 7   quarter_Quarter2             1197 non-null   bool   
 8   quarter_Quarter3             1197 non-null   bool   
 9   quarter_Quarter4             1197 non-null   bool   
 10  quarter_Quarter5             1197 non-null   bool   
 11  department_finishing         1197 non-null   bool   
 12  department_packing           1197 non-null   bool   
 13  department_sewing 

In [52]:
X.head()

,team,targeted_productivity,smv,idle_time,idle_men,no_of_style_change,quarter_Quarter1,quarter_Quarter2,quarter_Quarter3,quarter_Quarter4,...,incentiveRange_1309-2617,incentiveRange_2618-3271,incentiveRange_Above 3271,no_of_workersRange_2-11,no_of_workersRange_12-22,no_of_workersRange_23-33,no_of_workersRange_34-44,no_of_workersRange_45-55,no_of_workersRange_56-77,no_of_workersRange_Above 77
0,8,0.80,26.16,0.0,0,0,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,0.75,3.94,0.0,0,0,True,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2,11,0.80,11.41,0.0,0,0,True,False,False,False,...,False,False,False,False,False,True,False,False,False,False
3,12,0.80,11.41,0.0,0,0,True,False,False,False,...,False,False,False,False,False,True,False,False,False,False
4,6,0.80,25.90,0.0,0,0,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False


**<br>Check column names of numeric attributes**
<br>Features with numeric values need to be normalised by changing the values to
small numbers in a specific range (scaling). _Note that scaling comes_ after _the test//train split!_

In [53]:
numeri = X.select_dtypes(include=['float64','int64']).columns
print(numeri.to_list())

['team', 'targeted_productivity', 'smv', 'idle_time', 'idle_men', 'no_of_style_change']


***

**<br>Create Test // Train Datasets**
> Split X and y datasets into Train and Test subsets,<br>keeping relative proportions of each class (stratify)

In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =train_test_split(X, y,
                                                   test_size=0.2,
                                                   random_state=50)
# train_test_split does random selection, 
#      so we should reset the dataframe indexes
X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

***
Next are standard steps for all datasets: _scaling, classifiers, results_

**Scaling** comes _after_ test // train split

In [55]:
# data before normalization
X_test.head()

,team,targeted_productivity,smv,idle_time,idle_men,no_of_style_change,quarter_Quarter1,quarter_Quarter2,quarter_Quarter3,quarter_Quarter4,...,incentiveRange_1309-2617,incentiveRange_2618-3271,incentiveRange_Above 3271,no_of_workersRange_2-11,no_of_workersRange_12-22,no_of_workersRange_23-33,no_of_workersRange_34-44,no_of_workersRange_45-55,no_of_workersRange_56-77,no_of_workersRange_Above 77
0,6,0.75,2.90,0.0,0,0,False,True,False,False,...,False,False,False,True,False,False,False,False,False,False
1,1,0.65,3.94,0.0,0,0,True,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2,7,0.65,3.94,0.0,0,0,False,False,False,True,...,False,False,False,True,False,False,False,False,False,False
3,12,0.80,11.61,0.0,0,0,False,False,True,False,...,False,False,False,False,False,True,False,False,False,False
4,9,0.80,3.94,0.0,0,0,False,True,False,False,...,False,False,False,True,False,False,False,False,False,False


In [56]:
# scaling the Numeric columns 
# StandardScaler range: -1 to 1, MinMaxScaler range: zero to 1

# from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# sklearn docs say 
#   "Don't cheat - fit only on training data, then transform both"
#   fit() expects 2D array: reshape(-1, 1) for single col or (1, -1) single row

for i in numeri:
    arr = np.array(X_train[i])
    scale = MinMaxScaler().fit(arr.reshape(-1, 1))
    X_train[i] = scale.transform(arr.reshape(len(arr),1))

    arr = np.array(X_test[i])
    X_test[i] = scale.transform(arr.reshape(len(arr),1))
    

In [57]:
# data after normalization
X_test.head()

,team,targeted_productivity,smv,idle_time,idle_men,no_of_style_change,quarter_Quarter1,quarter_Quarter2,quarter_Quarter3,quarter_Quarter4,...,incentiveRange_1309-2617,incentiveRange_2618-3271,incentiveRange_Above 3271,no_of_workersRange_2-11,no_of_workersRange_12-22,no_of_workersRange_23-33,no_of_workersRange_34-44,no_of_workersRange_45-55,no_of_workersRange_56-77,no_of_workersRange_Above 77
0,0.454545,0.888889,0.000000,0.0,0.0,0.0,False,True,False,False,...,False,False,False,True,False,False,False,False,False,False
1,0.000000,0.666667,0.020132,0.0,0.0,0.0,True,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2,0.545455,0.666667,0.020132,0.0,0.0,0.0,False,False,False,True,...,False,False,False,True,False,False,False,False,False,False
3,1.000000,1.000000,0.168602,0.0,0.0,0.0,False,False,True,False,...,False,False,False,False,False,True,False,False,False,False
4,0.727273,1.000000,0.020132,0.0,0.0,0.0,False,True,False,False,...,False,False,False,True,False,False,False,False,False,False


***

***